In [1]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using HDF5
using DynamicPolynomials
using Plots
using Dates

using Optim
using Random

Consider simple Lindblad master equation with two dissipators:

$m
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\gamma_1\left[J_1 \rho J_1^\dagger - \frac{1}{2}\left\{ J_1^\dagger J_1, \rho \right\} \right]
     +\gamma_2\left[J_2 \rho J_2^\dagger - \frac{1}{2}\left\{ J_2^\dagger J_2, \rho \right\} \right]
$,

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} w & 0 \\ 0 & 0
   \end{pmatrix}
$

 withot loss of generality we can possibly look for jump operator of the form:

$
J_1 = \begin{pmatrix} 0 & a \\ 0 & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 & \sqrt{\gamma_1} \\ 0 & 0 
   \end{pmatrix}
$

$
J_2 = \begin{pmatrix} d & 0 \\ 0 & -d 
   \end{pmatrix} = 
   \begin{pmatrix} \sqrt{\gamma_2} & 0 \\ 0 & -\sqrt{\gamma_2} 
   \end{pmatrix}
$

In [2]:
# Define polynomial variables
@polyvar w r ϕ

Hˢʸᵐᵇ = [ w   0
          0   0. ]

J₁ˢʸᵐᵇ = [ 0   r
           0   0. ]

J₂ˢʸᵐᵇ = ϕ * [ 1    0
               0   -1. ]


function g_objective2(γᵢ)

    objₑₓ = 0

    for df_trn in train_files # loop over initial states

        ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_2CUT_data.h5", γᵢ)
        
        if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
            
        ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ[1:end_train])
        
        tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ[1:end_train])

        objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, [J₁ˢʸᵐᵇ, J₂ˢʸᵐᵇ])
        
    end # of files (initial states) loop

    return(objₑₓ)
end

g_objective2 (generic function with 1 method)

In [3]:

function scaling_poly(p::Polynomial)

    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling)) # 
end

function local_rand_min(p)

    pd = p / maximum(abs.(coefficients(p)))

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    num_iterations = 100

    # Initialize the best minimizer and the minimum value
    best_minimizer = nothing
    best_min_value = Inf

    num_of_variables = length(variables(pd))

    for _ in 1:num_iterations
        # Generate a random initial condition
        initial_point = rand(num_of_variables).*250

        # Run local optimization
        result = Optim.optimize(p_scaled, initial_point, BFGS())
        #println(Optim.minimum(result))

        # Update the best minimizer if a better one is found
        if Optim.minimum(result) < best_min_value
            
            best_minimizer = Optim.minimizer(result)
            best_min_value = Optim.minimum(result)
            
        end

    end

    best_minimizer = abs.(best_minimizer) # to make gamma positive

    minimizer_scaled = scale .* best_minimizer

    solution = variables(p_scaled) => minimizer_scaled

end

import Base.abs

function absm(M::AbstractMatrix)
    sqrt(M'*M)
end

function my_fidelity(ρ, σ)
    F = (tr(absm(sqrtm(ρ)*sqrtm(σ))))^2
    if imag(F) > 1e-3 throw(F, "Fidelity is complex number. Real part returned.") end
    real(F)
end

my_fidelity (generic function with 1 method)

In [4]:
data_dir = "../DATA/"
println(data_dir)

models_dir = "../MODELS/"
tests_dir = "../TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];

basis_files = ["State_B"*string(n) for n=1:4];

all_files = vcat(dodeca_files, basis_files)
train_files = basis_files 
test_files = dodeca_files;


../DATA/


In [5]:
γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

tests_data_file_name = "POP_LME_2_random_trn4_tst20_"*date_and_time_string * ".h5"

for γᵢ in γ

    println("γ =  "*γᵢ)

    poly = g_objective2(γᵢ)
    
    sol = local_rand_min(poly)

    omega = subs(w, sol)

    relaxation = subs(r^2, sol)
    de_phasing = subs(ϕ^2, sol)

    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol)
    J₁ˢⁱᵈ = subs(J₁ˢʸᵐᵇ, sol)
    J₂ˢⁱᵈ = subs(J₂ˢʸᵐᵇ, sol)

    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ) # create coupling group
        γ_group["omega"] = convert(Float64, omega)
        γ_group["gamma_relaxation"] = convert(Float64, relaxation)
        γ_group["gamma_de_phasing"] = convert(Float64,  de_phasing)
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)      
    end

    for df in test_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)

        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tˢⁱᵈ, ρˢⁱᵈ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈ), [DenseOperator(basis,J₁ˢⁱᵈ), DenseOperator(basis,J₂ˢⁱᵈ)])

        Fᵐʸ = [my_fidelity(ρ₁, ρ₂.data) for (ρ₁, ρ₂) in zip(ρₛ, ρˢⁱᵈ)]

        Fᴸᴹᴱₑₓ = [fidelity(ρ₁, ρ₂) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]                    

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, Fᴸᴹᴱₑₓ)
            init_state_group["My_Fidelity"] = convert.(Float64, Fᵐʸ)
            
        end
                    
        println(minimum(Fᴸᴹᴱₑₓ))
        println(minimum(Fᵐʸ))
    
    end

    println()
end

println(tests_data_file_name)

γ =  0.079477


State_D1 0.9993686233877914


0.9987376454120022
State_D2 

0.9993718508383901
0.9987440962481514
State_D3 0.9993717331291011


0.9987438609774633
State_D4 

...


0.9988507756416384
State_D16 0.9994252226362462
0.9988507756415115
State_D17 0.994164919120432
0.9883638864097358
State_D18 0.996249580633539
0.9925132269125031
State_D19 0.9993368312732994
0.998674102339358
State_D20 

0.9986509354637014
0.9973036909025259

γ =  251.33
State_D1 

0.9965366708487808
0.9930853363463713
State_D2 0.9951105957424201
0.9902450977588345
State_D3 0.9957958347779193
0.9916093445610525
State_D4 0.9944296429722385
0.9888903148218947
State_D5 0.9934695825480298
0.9869818114481584
State_D6 0.9934695825480298
0.9869818114481584
State_D7 0.9996723289110381
0.9993447651904263
State_D8 0.9982712017626757
0.9965453922686979
State_D9 

0.9947913973940544
0.9896099243292152
State_D10 0.993633708992111
0.9873079476454198
State_D11 0.9957958347779072
0.9916093445610287
State_D12 0.9944296429722385
0.9888903148218947
State_D13 0.9965366708487808
0.9930853363463713
State_D14 0.9951105957424836
0.9902450977589589
State_D15 0.9930260685753796
0.9861007728702742
State_D16 0.9930260685753329
0.9861007728701823
State_D17 0.9996723289110201
0.9993447651903863
State_D18 0.9982712017626716


0.9965453922686899
State_D19 0.993633708992111
0.9873079476454198
State_D20 0.9947913973939456
0.9896099243290002

POP_LME_2_random_trn4_tst20_2024-Jan-28_at_15-36.h5
